In [1]:
import sqlite3 as sql 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
conn = sql.connect('ETL.db')
cursor = conn.cursor()

In [2]:
df = pd.read_sql_query('SELECT * FROM Cotacao_euro', conn)
df

,moeda,nome,compra,venda,dataAtualizacao


In [3]:
url = 'https://br.dolarapi.com/v1/cotacoes'
response = requests.get(url)            
if response.status_code != 200:
    raise Exception('Erro ao acessar a API')
data = response.json()
data

[{'moeda': 'ARS',
  'nome': 'Peso Argentino',
  'compra': 0.0037,
  'venda': 0.0037,
  'fechoAnterior': 0.0037,
  'dataAtualizacao': '2026-01-22T12:02:12.303Z'},
 {'moeda': 'USD',
  'nome': 'Dólar',
  'compra': 5.311,
  'venda': 5.313,
  'fechoAnterior': 5.3196,
  'dataAtualizacao': '2026-01-21T00:00:00.000Z'},
 {'moeda': 'EUR',
  'nome': 'Euro',
  'compra': 6.22286,
  'venda': 6.23836,
  'fechoAnterior': 6.21826,
  'dataAtualizacao': '2026-01-22T12:03:12.000Z'},
 {'moeda': 'CLP',
  'nome': 'Peso Chileno',
  'compra': 0.00607,
  'venda': 0.006072,
  'fechoAnterior': 0.006076,
  'dataAtualizacao': '2026-01-22T12:00:51.070Z'},
 {'moeda': 'UYU',
  'nome': 'Peso Uruguayo',
  'compra': 0.1381,
  'venda': 0.1382,
  'fechoAnterior': 0.1385,
  'dataAtualizacao': '2026-01-22T12:00:51.070Z'}]

In [2]:
cursor.execute('''
               
      create table if not exists Cotacao_euro (         
               
             
                moeda text not null,
                nome text not null,
                compra real not null,
                venda real not null,
                dataAtualizacao text not null

              

               )
               
               ''')

In [4]:
cotacao_euro = []
for cotacao in data:
    if cotacao ['moeda'] == 'EUR':
        cotacao_euro.append({

            'moeda': cotacao['moeda'],
            'nome': cotacao['nome'],   
            'compra': cotacao['compra'],
            'venda': cotacao['venda'],
            'dataAtualizacao': cotacao['dataAtualizacao']



        })

In [5]:
for cotacao in cotacao_euro:
    cursor.execute('''INSERT INTO Cotacao_euro (moeda, nome, dataAtualizacao, compra, venda) 
                      VALUES (?, ?, ?, ?, ?)''', 
                   (cotacao['moeda'], cotacao['nome'], cotacao['dataAtualizacao'], cotacao['compra'], cotacao['venda']))

In [6]:
conn.commit()
conn.close()